# Data Loads and Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import pandas as pd

burnfit_dir = os.path.join("/content/drive/MyDrive", "Burnfit")
inputs = os.listdir(os.path.join(burnfit_dir, "input"))
outputs = os.listdir(os.path.join(burnfit_dir, "output"))

input_user_rm_csv = os.path.join(burnfit_dir, "input", "input_user_rm.csv")
input_user_csv = os.path.join(burnfit_dir, "input", "input_user.csv")
output_weight_rate_csv = os.path.join(burnfit_dir, "output", "output_increase_rate.csv")
output_weekly_increase_rate_plan_csv = os.path.join(burnfit_dir, "output", "output_weekly_increase_rate_plan.csv")

input_user_df = pd.read_csv(input_user_csv)
input_user_rm_df = pd.read_csv(input_user_rm_csv)
output_increase_rate_df = pd.read_csv(output_weight_rate_csv)
output_weekly_increase_rate_plan_df = pd.read_csv(output_weekly_increase_rate_plan_csv)
print(input_user_df.head())
print(input_user_rm_df.head())
print(output_increase_rate_df.head())
print(output_weekly_increase_rate_plan_df.head())

             id  운동경험  성별  운동목표
0  example_0000     1  남성     4
1  example_0001     2  남성     3
2  example_0002     4  남성     2
3  example_0003     1  남성     6
4  example_0004     1  남성     4
             id  벤치프레스    스쿼트  데드리프트 오버헤드프레스
0  example_0000   30kg   30kg   30kg    30kg
1  example_0001   60kg   90kg   70kg    30kg
2  example_0002  110kg  125kg  110kg    60kg
3  example_0003   30kg   30kg   30kg    30kg
4  example_0004   30kg   30kg   30kg    30kg
             id  init_weight_rate  increase_rate_week  increase_rate_set  \
0  example_0000                45                  10                  5   
1  example_0001                55                  10                  5   
2  example_0002                50                  10                  5   
3  example_0003                55                   5                 10   
4  example_0004                65                  10                  5   

   deloading_rate  
0              50  
1              30  
2              60  
3

# Weekly Increase Rate Plan Generation

In [ ]:
def weekly_increase_rate_plan_generator(init_weight_rate, increase_rate_week, increase_rate_set, deloading_rate):
  current_weight = init_weight_rate
  week1 = f"Week1\nSET1 {current_weight}%x5\nSET2 {current_weight+increase_rate_set}%x5\nSET3 {current_weight+2*increase_rate_set}%x5+\n"
  current_weight+=increase_rate_week
  week2 = f"\nWeek2\nSET1 {current_weight}%x3\nSET2 {current_weight+increase_rate_set}%x3\nSET3 {current_weight+2*increase_rate_set}%x3+\n"
  current_weight+=increase_rate_week
  week3 = f"\nWeek3\nSET1 {current_weight}%x5\nSET2 {current_weight+increase_rate_set}%x3\nSET3 {current_weight+2*increase_rate_set}%x1+\n"
  week4 = f"\nWeek4\nSET1 {deloading_rate}%x5\nSET2 {deloading_rate+increase_rate_set}%x5\nSET3 {deloading_rate+2*increase_rate_set}%x5\n"
  return week1+week2+week3+week4

In [ ]:
weekly_increase_rate_plan_df = pd.DataFrame(columns=["id", "weekly_increase_rate_plan"])
weekly_increase_rate_plan_df["id"] = output_increase_rate_df["id"]

for idx, row in output_increase_rate_df.iterrows():
  weekly_increase_rate_plan_df.loc[idx, "weekly_increase_rate_plan"] = weekly_increase_rate_plan_generator(row["init_weight_rate"],
                                                                                                           row["increase_rate_week"],
                                                                                                           row["increase_rate_set"],
                                                                                                           row["deloading_rate"])
output_weekly_increase_rate_plan_csv = os.path.join(burnfit_dir, "output", "output_weekly_increase_rate_plan.csv")
weekly_increase_rate_plan_df.to_csv(output_weekly_increase_rate_plan_csv, index=False)

# Json Generator


In [ ]:
training_dir = os.path.join(burnfit_dir, "training_dir")
training_jsonl = os.path.join(burnfit_dir, "training_v1.jsonl")
print(training_jsonl)

combined_df = input_user_df.merge(input_user_rm_df, on="id") \
            .merge(output_increase_rate_df, on="id") \
            .merge(output_weekly_increase_rate_plan_df, on="id")

print(combined_df.columns)

with open(training_jsonl, "w", encoding="utf-8") as f:
  for _, row in combined_df.iterrows():
    input_dict = {
        "운동경험": row["운동경험"],
        "성별": row['성별'],
        "운동목표": row['운동목표'],
        "1RM": {
            "벤치프레스": row['벤치프레스'],
            "스쿼트": row['스쿼트'],
            "데드리프트": row['데드리프트'],
            "오버헤드프레스": row['오버헤드프레스']
        }
    }

    output_dict = {
        "init_weight_rate":row["init_weight_rate"],
        "increase_rate_week":row["increase_rate_week"],
        "increase_rate_set":row["increase_rate_set"],
        "deloading_rate":row["deloading_rate"],
        "weekly_increase_rate_plan":row["weekly_increase_rate_plan"]
    }

    text = f"<INSTRUCTION> 주어진 내용을 바탕으로 4주간의 무게 증량 계획을 생성하세요. " \
                f"<INPUT> {json.dumps(input_dict, ensure_ascii=False)} " \
                f"<OUTPUT> {json.dumps(output_dict, ensure_ascii=False)} <EOS>"
    f.write(json.dumps({"text": text}, ensure_ascii=False) + "\n")
    print("Index: ", _)
    print(text)